# Optimal transfer learning

Here, we create two models and traverse a path on the statistical manifold between them.
Given my theory's optimal infinitesimal traversals of continuously changing distributions, 
the theory should port cleanly from reinforcement learning to transfer learning. 
Of course, only each infinitesimal step along the path with be optimal, 
leaving room for data scientists' intuition to usefully bias retention. 
For example, if early game information only becomes useful late in play, 
then a retention spike will need to be added. 
Regardless, infinitesimal optimality is an important step toward a deep and coherent transfer learning theory. 

The experiment will involve fitting a dense net to the MNIST dataset, classifying digits as usual. 
However, after the initial fit, we will embed the dense net into a much larger statistical manifold 
by making it part of a mixture model with a different, far more sparse model. 
We'll then slowly and continuously traverse the mixture probability parameter $q$ from 0 to 1, 
optimally retaining information at each step. 
We'll measure test set accuracy at each step and compare against different traversal strategies. 
A good result should find accuracy is optimally and usefully sustained despite transferring to a sparse model.

If successful, this experiment will illustrate the effectiveness of my general theory of transfer learning. 
Thus, it'd be possible to optimally translate information optimally between arbitrary models, 
provided they can handle the same dataset or a continuously transforming dataset between the models.
This is a powerful result, so I'm eager to run my experiment, but won't get my hopes up too much. 
The truth is best found by the pursuit of ambitious targets and sensitively understanding negative results.

In [1]:
## Dense net code initially authored by Google's search engine GenAI on 20 Oct 2024. 
## I've applied minor modifications for generality, but the code worked great on first draft. 

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the DenseNet model
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.features = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.features(x)
        return x

# Load MNIST dataset
train_dataset = datasets.MNIST(root='/tmp/data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='/tmp/data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model, loss function and optimizer
model = DenseNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for i, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/autograd/graph.py:769: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/5], Step [1/938], Loss: 2.2946
Epoch [1/5], Step [101/938], Loss: 0.3633
Epoch [1/5], Step [201/938], Loss: 0.1715
Epoch [1/5], Step [301/938], Loss: 0.2084
Epoch [1/5], Step [401/938], Loss: 0.3679
Epoch [1/5], Step [501/938], Loss: 0.2522
Epoch [1/5], Step [601/938], Loss: 0.1443
Epoch [1/5], Step [701/938], Loss: 0.1262
Epoch [1/5], Step [801/938], Loss: 0.1290
Epoch [1/5], Step [901/938], Loss: 0.1676
Epoch [2/5], Step [1/938], Loss: 0.0586
Epoch [2/5], Step [101/938], Loss: 0.1659
Epoch [2/5], Step [201/938], Loss: 0.0710
Epoch [2/5], Step [301/938], Loss: 0.1123
Epoch [2/5], Step [401/938], Loss: 0.0921
Epoch [2/5], Step [501/938], Loss: 0.0941
Epoch [2/5], Step [601/938], Loss: 0.1603
Epoch [2/5], Step [701/938], Loss: 0.0231
Epoch [2/5], Step [801/938], Loss: 0.0420
Epoch [2/5], Step [901/938], Loss: 0.1315
Epoch [3/5], Step [1/938], Loss: 0.0339
Epoch [3/5], Step [101/938], Loss: 0.0512
Epoch [3/5], Step [201/938], Loss: 0.0050
Epoch [3/5], Step [301/938], Loss: 0.069

In [4]:
## Models for transfer learning to sparse nets 
from ssr_agent import SSRAgent 
from replay_buffer import ReplayBuffer 

class SparseActivation(nn.Module): 
    '''Softmax that zeros-out small values. 
    Transfer-learning from a prefit model recommended. 
    '''
    def __init__(self, K=1, next_linearity=None):
        'K: determines maximum number of non-zero dimensions.'
        super(SparseActivation, self).__init__()
        self.K = K 
        self.softmax = nn.Softmax(dim=1) 
        self.next_linearity = next_linearity 
        pass 
    def forward(self, x, next_linearity=None): 
        '''inputs:
        - x: [n, ...]-shaped tensor 
        - next_linearity: instance of nn.Linear is applied efficiently to output of this activation. Autodetects next_nonlinearity if it was provided during init. 
        ouputs: 
        - idx: indicies of softmax(x) that are greater than 1/K. 
        - softmax(x)[idx]: values of softmax(x) that are greater than 1/K. Returns None if idx is empty. 
        '''
        ## calculate 
        x = self.softmax(x) 
        idx = (x > 1/self.K).nonzero() 
        x_idx = None 
        if int(idx.shape[0]) > 0: 
            x_idx = x[idx] 
        if next_linearity is None and self.next_linearity is None: 
            return idx, x_idx 
        if next_linearity is None and self.next_linearity is not None: 
            next_linearity = self.next_linearity 
            pass 
        ## sparsely apply next_linearity 
        if int(idx.shape[0]) > 0: 
            y = x_idx.matmul(next_linearity.weights[idx,:]) 
        else: 
            y = 0. 
            pass 
        y += next_linearity.bias 
        return idx, x_idx, y 
    pass 

class SparseNet(nn.Module): 
    def __init__(self, K=10): 
        super(SparseNet, self).__init__() 
        self.K = K 
        self.linear1 = nn.Linear(784, 512) 
        self.linear2 = nn.Linear(512, 256) 
        self.linear3 = nn.Linear(256, 128) 
        self.linear4 = nn.Linear(128, 10) 
        self.activation1 = SparseActivation(K=self.K, next_linearity=self.linear2) 
        self.activation2 = SparseActivation(K=self.K, next_linearity=self.linear3) 
        self.activation3 = SparseActivation(K=self.K, next_linearity=self.linear4) 
        self.activation4 = nn.Softmax(dim=1) 
        pass 
    def forward(self, x): 
        x = self.activation1(self.linear1(x)) 
        x = self.activation2(self.linear2(x)) 
        x = self.activation3(self.linear3(x)) 
        x = self.activation4(self.linear4(x)) 
        return x

class MixtureModel(nn.Module): 
    'join two models with a Bernoulli random variable'
    def __init__(self, model1, model2, p): 
        'mix models 1 and 2, selecting model 2 with probability p'
        super(MixtureModel, self).__init__() 
        self.model1 = model1 
        self.model2 = model2 
        self.p = p 
        pass 
    def forward(self, x): 
        n = x.shape[0] 
        b = torch.rand(n) >= self.p 
        b_sum = b.sum() 
        y = torch.zeros([n,1]) 
        if b_sum < n: 
            y[b.logical_not()] = self.model1(x[b.logical_not(),:]) 
        if b_sum > 0: 
            y[b] = self.model2(x[b,:]) 
            pass 
        return y 
    pass 

class Model(SSRAgent): 
    'Run transfer learning experiments with this model' 
    def __init__(self, p, K=10, ssr_rank=5): 
        'p: probability of the mixture model selecting the sparse model' 
        super(Model, self).__init__(replay_buffer=ReplayBuffer(), ssr_rank=ssr_rank) 
        self.mixture_model = MixtureModel( 
            model1 = DenseNet(), 
            model2 = SparseNet(K=K), 
            p=p 
        ) 
        self.criterion = nn.CrossEntropyLoss() 
        self.optimizer = optim.Adam(self.parameters(), lr=0.001) 
        pass 
    def loss(self, data): 
        y_hat = self.mixture_model(data.x) 
        return self.criterion(y_hat, data.y) 
    def fit_iters(self, n_iters=100, batch_size=256, pi_max=.9): 
        model.train() 
        pi = model.optimal_lambda(pi_max=pi_max) 
        for _ in range(n_iters): 
            self.optimizer.zero_grad() 
            data = self.replay_buffer.sample(256) 
            loss = pi * self.loss(data) + (1 - pi) * self.ssr() 
            loss.backward() 
            optimizer.step() 
            pass 
        return pi  
    def eval(self): 
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in test_loader:
                output = self(x)
                _, y_hat = torch.max(output.data, 1)
                total += y.size(0)
                correct += (y_hat == y).sum().item() 
                pass 
            pass 
        return correct / total 
    pass 


In [5]:
## heavier compute starts here 
from tqdm import tqdm 

N = 100 ## number of path steps over p 

model = Model(p=0.) 

## load data 
for x, y in train_loader: 
    model.replay_buffer.add(x=x, y=y) 
    pass 

## traverse p 
acc = [] 
pi = [] 
tq = tqdm(range(N+1)) 
for i in tq: 
    model.p = i/N 
    n_iters = 100 if i == 0 else 10 
    p = model.fit_iters(n_iters=n_iters) 
    a = model.eval() 
    acc.append(a) 
    pi.append(p) 
    tq.set_description(f'acc: {round(a,3)}, pi: {round(p, 3)}') 
    pass 

  0%|          | 0/101 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (7168x28 and 784x512)